## <center> <font color='purple'> Dynamic Ensemble Machine Learning Models (Single ML Pool)</font></center> 
#### <center>Firuz Juraev (Sungkyunkwan Unniversity)</center>

 ####  <font color='purple'>In this notebook, we will test Dynamic Ensemble ML models with all single ML classifiers 

### <font color='green'> Libraries 

#### <font color='blue'> Basic Libraries 

In [1]:
import pandas as pd
import numpy as np 
from numpy import mean
from numpy import std
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### <font color='blue'> Single ML Models Libraries 

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#### <font color='blue'> Static ML Models Libraries 

In [3]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#### <font color='blue'> DES Libraries 

In [4]:
from deslib.des import DESP
from deslib.des import KNORAE
from deslib.des import KNORAU
from deslib.des import METADES
from deslib.des import DESKNN # new 
from deslib.des import KNOP # new 

#### <font color='blue'> DCS Libraries 

In [5]:
from deslib.dcs import MCB

#### <font color='blue'> Processing Libraries 

In [6]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score, 
                             f1_score,
                             roc_auc_score, 
                             auc)
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### <font color='green'> Load Dataset 

In [7]:
train_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_train_data_4.csv")
test_dataset_upsampled = pd.read_csv("Data/Resampled_neonates_test_data_4.csv")

X_train = train_dataset_upsampled.drop(["DEAD"], axis=1) 
y_train = train_dataset_upsampled["DEAD"]

X_test = test_dataset_upsampled.drop(["DEAD"], axis=1) 
y_test = test_dataset_upsampled["DEAD"]

In [8]:
columns = X_train.columns.to_list()

### <font color='green'> Data Normalization

In [9]:
Min_max_scaler = MinMaxScaler().fit(X_train)

## Scaling 
X_train_mm_scaled = Min_max_scaler.transform(X_train)
X_test_mm_scaled = Min_max_scaler.transform(X_test)

## Numpy Array to DataFrame 
df_train_mm_scaled = pd.DataFrame(X_train_mm_scaled, columns = columns)
df_test_mm_scaled = pd.DataFrame(X_test_mm_scaled, columns = columns)

### <font color='green'> Feature Selection

In [10]:
tain_mm_scaled_df = df_train_mm_scaled[:]
tain_mm_scaled_df["DEAD"] = y_train 

test_mm_scaled_df = df_test_mm_scaled[:] 
test_mm_scaled_df["DEAD"] = y_test


importances = mutual_info_classif(df_train_mm_scaled, y_train)
feat_importance = pd.Series(importances, tain_mm_scaled_df.columns[0:len(tain_mm_scaled_df.columns)-1])
    
feat_importance = feat_importance.sort_values(ascending=False)
    
selected_features = feat_importance[:30]
selected_features_list_mm_scaled = selected_features.index.to_list()


tain_mm_scaled_df[selected_features_list_mm_scaled].head(2)

,temperature_mean,respRate_std,respRate_var,skinTemperature_std,skinTemperature_var,heartRate_std,heartRate_var,sao2_std,bpCuffMean_var,sao2_var,...,BIRTH_WEIGHT,bpCuffSystolic_mean,bpCuffDiastolic_mean,sao2_mean,temperature_var,glucometer_mean,temperature_std,bpCuffMean_mean,PLATELET,D10W_MEAN
0,0.775318,0.422245,0.178291,0.134078,0.017977,0.22911,0.057937,0.097817,0.053294,0.009568,...,0.264267,0.602649,0.457143,0.881455,0.015695,0.312593,0.125281,0.310696,0.281330,0.151138
1,0.709668,0.583964,0.341014,0.086672,0.007512,0.28258,0.086102,0.080938,0.020317,0.006551,...,0.437819,0.644907,0.479330,0.926056,0.003770,0.272551,0.061399,0.351928,0.392157,0.196784


In [11]:
X_train_mm = df_train_mm_scaled[selected_features_list_mm_scaled][:]
X_test_mm = df_test_mm_scaled[selected_features_list_mm_scaled][:]

### <font color='purple'> Hold-out Test (With Single ML) - (+FS, +HO)

In [12]:
def hold_out_single_ML(): 
    rng = np.random.RandomState(42) 
    X_train, X_dsel, y_train_en, y_dsel = train_test_split(X_train_mm, y_train, test_size=0.40, random_state=rng)
    
    model_svc = SVC(kernel='linear', C=0.007, gamma=0.2, degree=3, probability=True, class_weight='balanced')
    model_dt = DecisionTreeClassifier(criterion='entropy', max_depth=3)
    model_lr = LogisticRegression(penalty='l2', C=0.002)
    model_ml_perceptron = MLPClassifier(solver='adam', max_iter=11, verbose=10,learning_rate_init=.003)
    model_nb = GaussianNB(var_smoothing=0.1)
    model_knn = KNeighborsClassifier(n_neighbors=5, leaf_size=30, p=2)
    
    model_svc.fit(X_train, y_train_en)
    model_dt.fit(X_train, y_train_en)
    model_lr.fit(X_train, y_train_en)
    model_ml_perceptron.fit(X_train, y_train_en)
    model_nb.fit(X_train, y_train_en) 
    model_knn.fit(X_train, y_train_en) 
    
    pool_classifiers = [model_ml_perceptron, 
                        model_dt,
                        model_lr, 
                        model_svc, 
                        model_nb, 
                        model_knn] 
    
    knorau = KNORAU(pool_classifiers)
    kne = KNORAE(pool_classifiers)  
    metades = METADES(pool_classifiers, k=12)
    desknn = DESKNN(pool_classifiers)
    mcb = MCB(pool_classifiers)
    desp = DESP(pool_classifiers)
    knop = KNOP(pool_classifiers, k=12)

    fire_knorau = KNORAU(pool_classifiers, DFP=True, k=7)
    fire_kne = KNORAE(pool_classifiers, DFP=True, k=9) 
    fire_metades = METADES(pool_classifiers, DFP=True, k=15)
    fire_desknn = DESKNN(pool_classifiers, DFP=True, k=9)
    fire_mcb = MCB(pool_classifiers, DFP=True, k=9) # 9 was 
    fire_desp = DESP(pool_classifiers, DFP=True, k=9)
    fire_knop = KNOP(pool_classifiers, DFP=True, k=15)
    
    ensemble_classifiers = [fire_knorau, knorau, fire_kne, kne, fire_metades, metades, fire_desknn, desknn, 
                            fire_mcb, mcb, fire_desp, desp, fire_knop, knop]
    
    ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                      "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]
    
    acc_list = [] 
    precision_list = [] 
    recall_list = []
    f1_lists = [] 
    auc_list = [] 
    fpr_list = [] 
    tpr_list = []
    
    for e_cls in ensemble_classifiers:
        e_cls.fit(X_dsel, y_dsel)
        y_preds = e_cls.predict(X_test_mm) 
        yproba = e_cls.predict_proba(X_test_mm)[::,1] 

        acc_list.append(accuracy_score(y_test, y_preds))
        precision_list.append(precision_score(y_test, y_preds))
        recall_list.append(recall_score(y_test, y_preds))
        fone = f1_score(y_test, y_preds)
        f1_lists.append(fone) 
        
        fpr, tpr, _ = roc_curve(y_test,  yproba)
        fpr_list.append(fpr)
        tpr_list.append(tpr)
        auc = roc_auc_score(y_test, yproba)
        auc_list.append(auc) 
    
    results =  {'name': ensemble_names, 
                'accuracy': acc_list, 
                'precision': precision_list, 
                'recall': recall_list, 'f1': f1_lists, 
                'auc': auc_list, 
                'tpr': tpr_list, 'fpr': fpr_list}
    
    df = pd.DataFrame.from_dict(results)
    
    return df

In [13]:
results_data = []
for i in range(0, 10):
    result = hold_out_single_ML()
    results_data.append(result)
    

fireResultsDF = pd.concat(results_data)

Iteration 1, loss = 0.63480459
Iteration 2, loss = 0.52007425
Iteration 3, loss = 0.43741296
Iteration 4, loss = 0.39052832
Iteration 5, loss = 0.36159637
Iteration 6, loss = 0.33890135
Iteration 7, loss = 0.33226139
Iteration 8, loss = 0.31802526
Iteration 9, loss = 0.30612113
Iteration 10, loss = 0.29438921
Iteration 11, loss = 0.28702294
Iteration 1, loss = 0.64955393
Iteration 2, loss = 0.54312741
Iteration 3, loss = 0.45255652
Iteration 4, loss = 0.39094851
Iteration 5, loss = 0.35731017
Iteration 6, loss = 0.33595477
Iteration 7, loss = 0.32013254
Iteration 8, loss = 0.30530648
Iteration 9, loss = 0.29454854
Iteration 10, loss = 0.28905090
Iteration 11, loss = 0.28233558
Iteration 1, loss = 0.65294212
Iteration 2, loss = 0.54784253
Iteration 3, loss = 0.45939909
Iteration 4, loss = 0.40093278
Iteration 5, loss = 0.36686809
Iteration 6, loss = 0.34343167
Iteration 7, loss = 0.32922493
Iteration 8, loss = 0.31459364
Iteration 9, loss = 0.30380424
Iteration 10, loss = 0.29330614
Ite

In [14]:
ensemble_names = ["FIRE-KNORA-U", "KNORA-U", "FIRE-KNORA-E", "KNORA-E", "FIRE-METADES", "METADES",
                  "FIRE-DESKNN", "DESKNN", "FIRE-MCB", "MCB", "FIRE-DESP", "DESP", "FIRE-KNOP", "KNOP"]

accuracy = [] 
accuracy_std =  [] 
precision = [] 
precision_std = [] 
recall = [] 
recall_std = [] 
f1_score = [] 
f1_std = []
auc_score = []
auc_std = []


for n in ensemble_names:
    accuracy.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.mean(), 3))
    accuracy_std.append(round(fireResultsDF[fireResultsDF.name == n].accuracy.std(), 3))
    precision.append(round(fireResultsDF[fireResultsDF.name == n].precision.mean(), 3))
    precision_std.append(round(fireResultsDF[fireResultsDF.name == n].precision.std(), 3))
    recall.append(round(fireResultsDF[fireResultsDF.name == n].recall.mean(), 3))
    recall_std.append(round(fireResultsDF[fireResultsDF.name == n].recall.std(), 3))
    f1_score.append(round(fireResultsDF[fireResultsDF.name == n].f1.mean(), 3))
    f1_std.append(round(fireResultsDF[fireResultsDF.name == n].f1.std(), 3))
    auc_score.append(round(fireResultsDF[fireResultsDF.name == n].auc.mean(), 3))
    auc_std.append(round(fireResultsDF[fireResultsDF.name == n].auc.std(), 3))
    
final_results = {"method": ensemble_names, 
                     "accuracy": accuracy, 
                     "accuracy_std": accuracy_std,
                     "precision": precision, 
                     "precision_std": precision_std,
                     "recall": recall, 
                     "recall_std": recall_std,
                     "f1_score": f1_score, 
                     "f1_std": f1_std, 
                     "auc": auc_score, 
                     "auc_std": auc_std}

finalResultsDF = pd.DataFrame.from_dict(final_results)

In [15]:
finalResultsDF

,method,accuracy,accuracy_std,precision,precision_std,recall,recall_std,f1_score,f1_std,auc,auc_std
0,FIRE-KNORA-U,0.982,0.001,0.966,0.001,1.000,0.000,0.983,0.001,0.994,0.000
1,KNORA-U,0.978,0.001,0.958,0.002,1.000,0.000,0.979,0.001,0.994,0.000
2,FIRE-KNORA-E,0.919,0.000,0.977,0.001,0.859,0.000,0.914,0.000,0.991,0.000
3,KNORA-E,0.921,0.000,0.981,0.001,0.859,0.000,0.916,0.000,0.991,0.000
4,FIRE-METADES,0.917,0.018,0.975,0.002,0.856,0.036,0.911,0.021,0.985,0.003
5,METADES,0.913,0.017,0.980,0.004,0.843,0.032,0.906,0.020,0.972,0.027
6,FIRE-DESKNN,0.914,0.000,0.989,0.001,0.838,0.000,0.907,0.000,0.989,0.000
7,DESKNN,0.958,0.000,0.990,0.001,0.924,0.000,0.956,0.000,0.992,0.000
8,FIRE-MCB,0.940,0.005,0.975,0.002,0.904,0.010,0.938,0.006,0.905,0.004
9,MCB,0.941,0.003,0.973,0.004,0.906,0.006,0.939,0.004,0.906,0.009


In [16]:
# finalResultsDF.to_csv("Results/single_pool_results/des_single_pool_results_6_models.csv", index=False)